<a href="https://colab.research.google.com/github/agmCorp/colab/blob/main/solucion3_matching_catalogos_bse_autodata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Matching Optimizado v3: BSE vs Autodata


## 1️⃣ Instalación de dependencias

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Instalar librerías necesarias
!pip install -q sentence-transformers pandas numpy unidecode tqdm rank_bm25

## 2️⃣ Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from unidecode import unidecode
import re
from tqdm.auto import tqdm
import warnings
import torch
from collections import defaultdict

warnings.filterwarnings('ignore')
print("Librerías importadas correctamente")
print(f"GPU disponible: {torch.cuda.is_available()}")

## 3️⃣ Cargar los archivos CSV

In [ ]:
# Cargar los datos (datatype str para evitar problemas de lectura)
df_bse = pd.read_csv(
    '/content/sample_data/RAW_CART_MATRICERO_BSE_202602131851.csv',
    dtype=str,
    keep_default_na=False
 )
df_autodata = pd.read_csv(
    '/content/sample_data/RAW_CART_PAD_AUTODATA_202602131849.csv',
    dtype=str,
    keep_default_na=False
 )

print(f"Registros BSE: {len(df_bse):,}")
print(f"Registros Autodata: {len(df_autodata):,}")
print("\nPrimeras filas de BSE:")
display(df_bse.head(10))
print("\nPrimeras filas de Autodata:")
display(df_autodata.head(10))

## 4️⃣ Funciones de Preprocesamiento

In [ ]:
# Caracteres extraños
def detectar_caracteres_especiales(df, columna):
    todos_los_textos = " ".join(df[columna].astype(str).unique())
    # Buscamos cualquier cosa que no sea letra, número, espacio o puntuación básica
    caracteres_raros = set(re.findall(r'[^a-zA-Z0-9\s.,\-\/]', todos_los_textos))
    return caracteres_raros

print("Caracteres extraños en BSE:", detectar_caracteres_especiales(df_bse, 'MODELO_ORIGINAL'))
print("Caracteres extraños en Autodata:", detectar_caracteres_especiales(df_autodata, 'MODELO_A_ORIGINAL'))

def estandarizar_terminos_tecnicos(texto):
    """Normaliza sinónimos técnicos a tokens estándar únicos ."""
    if not texto: return ""
    texto_procesado = texto.lower()

    # Expandir abreviaturas con slash frecuentes
    texto_procesado = re.sub(r'\bc\s*/\s*', ' con ', texto_procesado)
    texto_procesado = re.sub(r'\bp\s*/\s*', ' para ', texto_procesado)

    mapeos = {
        r"\baire\s+acondicionado\b|\baa\b|\ba/a\b|\baire\b|\bcon aire\b": " aire_acondicionado ",
        r"\bclimatizador\b|\bclim\b|\bclimaut\b": " climatizador ",
        r"\bpas\b|\bpax\b|\bpasajeros\b": " pasajeros ",
        r"\bhb\b|\bhatchback\b": " hatchback ",
        r"\bsb\b|\bsportback\b": " sportback ",
        r"\be\.full\b|\bex\.full\b|\bextra full\b": " extra_full ",
        r"\bs\.full\b|\bsuper full\b": " super_full ",
        r"\bcue\b|\bcuero\b": " cuero ",
        r"\btech\b|\btecho\b|\bt\.cielo\b": " techo_solar ",
        r"\bmb\b|\bmultim\b|\bmultimedia\b": " multimedia ",
        r"\bay\.est\b|\bay\.estac\b|\ba\.estac\b|\bp\.assi\b|\bp\.assist\b|\bpark assist\b": " ayuda_estacionamiento ",
        r"\bllan\b|\bllantas\b": " llantas ",
        r"\btd\b|\bt\.diesel\b|\bturbo d\b": " turbo_diesel ",
        r"\btdi\b|\btd intercooler\b|\bt\.diesel intercooler\b": " turbo_diesel_intercooler ",
        r"\btiptronic\b|\bs-tronic\b|\baut\.\b|\baut\b|\bautomatico\b": " automatico ",
        r"\b3-5p\b|\b3-5 ptas\b|\b3-5 puertas\b": " 3_5_puertas ",
        r"\b4-5p\b|\b4-5 ptas\b|\b4-5 puertas\b": " 4_5_puertas ",
        r"(?<!-)(?:\b2p\b|\b2 ptas\b|\b2 puertas\b)": " 2_puertas ",
        r"(?<!-)(?:\b3p\b|\b3 ptas\b|\b3 puertas\b)": " 3_puertas ",
        r"(?<!-)(?:\b4p\b|\b4 ptas\b|\b4 puertas\b)": " 4_puertas ",
        r"(?<!-)(?:\b5p\b|\b5 ptas\b|\b5 puertas\b)": " 5_puertas ",
        r"\badaptado a rural\b|\badaptada a rural\b|\bad\.?\s*rural\b": " furgon_reformado ",
        r"\blgo\b|\blargo\b": " largo ",
    }
    for patron, reemplazo in mapeos.items():
        texto_procesado = re.sub(patron, reemplazo, texto_procesado)
    return texto_procesado

def normalizar_texto(texto):
    """Repara caracteres extraños, unifica sinónimos y limpia símbolos."""
    if pd.isna(texto):
        return ""

    texto = str(texto).lower()

    # 1. Reparación de caracteres extraños comunes
    # IMPORTANTE: las secuencias multi-carácter (ã¡, ã©, ã³, ãº, ã±) deben ir
    # ANTES que "ã" sola, para evitar que "ã" consuma el primer carácter de la secuencia.
    MAPA_REPARACION = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ñ": "n",
        "ã¡": "a",
        "ã©": "e",
        "ã³": "o",
        "ãº": "u",
        "ã±": "n",
        "ã": "i",
        "¿": "o",
        "±": "n",
        "ý": "i",
        "ß": "a",
        "ð": "o",
        "â": "a",
        "²": "2",
        "³": "3",
        "¬": "",
        "·": "",
        "°": "",
        "º": "",
        "*": "",
        "+": "",
        "|": "",
        ")": "",
        '"': "",
        "!": "",
        "]": "",
        "[": "",
        "(": "",
        "?": "",
        "=": "",
    }

    for error, correccion in MAPA_REPARACION.items():
        texto = texto.replace(error, correccion)

    # 2. Unidecode y Estandarización
    texto = unidecode(texto)
    texto = estandarizar_terminos_tecnicos(texto)

    # 3. Marcadores temporales para preservar contexto
    decimal_token = "_decimal_tok_"
    hyphen_token = "_hyphen_tok_"

    # Preservar separadores decimales (ej: 1.5, 2,0)
    texto = re.sub(r"(?<=\d)[\.,](?=\d)", decimal_token, texto)
    # Preservar guiones válidos entre alfanuméricos (códigos y rangos: x-line, s-tronic)
    texto = re.sub(r'(?<=[a-z0-9])-(?=[a-z0-9])', hyphen_token, texto)


    # Expandir slash técnico entre alfanuméricos a '_' (dx/lx -> dx_lx)
    texto = re.sub(r'(?<=[a-z0-9])/(?=[a-z0-9])', '_', texto)

    # 4. Separar solo guión editorial (cuando aparece con espacios: 'full - extrafull')
    texto = re.sub(r'\s+-\s+', ' ', texto)

    # 5. Limpieza final: solo letras, números y guion bajo
    texto = re.sub(r"[^a-z0-9_\s]", " ", texto)

    # 6. Restaurar marcadores temporales
    texto = texto.replace(hyphen_token, '-')
    texto = texto.replace(decimal_token, '.')
    return re.sub(r"\s+", " ", texto).strip()

def corregir_anio_bse(anio):
    """Corrige años > 3000 restándoles 1000 (ej: 3020 -> 2020)"""
    if anio is None: return None
    return anio - 1000 if anio >= 3000 else anio

def extraer_anio_inicio_fin(rango_anios):
    if pd.isna(rango_anios): return None, None
    rango_str = str(rango_anios).strip()

    # Intenta encontrar dos años (rango)
    match = re.findall(r'(\d{4})', rango_str)
    if len(match) >= 2:
        return corregir_anio_bse(int(match[0])), corregir_anio_bse(int(match[1]))
    elif len(match) == 1:
        anio = corregir_anio_bse(int(match[0]))
        return anio, anio
    return None, None

def normalizar_marca(texto):
    if pd.isna(texto): return ""
    texto = str(texto).lower()
    # Elimina cualquier contenido entre picos <...>
    texto = re.sub(r'<.*?>', '', texto)
    return texto.strip()

# ========== TABLA DE ALIAS DE MARCAS ==========
# Mapea marca normalizada de Autodata → lista de marcas normalizadas en BSE.
# Soporta mapeo uno-a-muchos para marcas compuestas (ej: DONGFENG → busca en 2 marcas BSE).
# Si la marca Autodata no está en este diccionario, se usa tal cual (match directo).
ALIAS_MARCAS = {
    # --- Espaciado diferente ---
    "polarsun":         ["polar sun"],
    "routebuggy":       ["route buggy"],
    "zhong tong":       ["zhongtong"],
    "zxauto":           ["zx auto"],
    "tongbao":          ["tong bao"],
    "leapmotor":        ["leap motor"],
    # --- Nombre extendido / abreviado ---
    "kia motors":       ["kia"],
    "byd auto":         ["byd"],
    "bjc":              ["bjc-beijing"],
    "gwm":              ["gwm - [great wall motor]"],
    "gac - trumpchi":   ["gac"],
    "lynk co":          ["lynk y co"],
    "porsche replica":  ["porsche"],
    "brilliance sunra": ["brilliance"],
    # --- Typos en BSE (no se pueden corregir en CSV fuente) ---
    "shineray":         ["shinerai"],
    "fuqi":             ["fuqui"],
    # --- Marca compuesta → una marca BSE ---
    "dfsk":             ["dfm - dfsk (dong feng)"],
    "chana":            ["chana-changan"],
    "changan":          ["chana-changan"],
    "geely riddara":    ["riddara (geely)"],
    # --- Marca compuesta → múltiples marcas BSE ---
    "dongfeng":         ["dfm - dfsk (dong feng)", "forthing (dongfeng)"],
    "daewoo - fso":     ["daewoo", "fso"],
    "vaz 1111 / uaz":   ["vaz", "uaz"],
}

def resolver_marcas_bse(marca_auto_norm):
    """Resuelve una marca normalizada de Autodata a lista de marcas BSE equivalentes."""
    if marca_auto_norm in ALIAS_MARCAS:
        return ALIAS_MARCAS[marca_auto_norm]
    return [marca_auto_norm]

def validar_normalizacion_contextual(df_bse, df_autodata, n_muestra=50):
    """Genera validación rápida con tabla de ejemplos y muestra real para revisión manual."""
    ejemplos = [
        ("c/diesel 2,0", "con diesel 2.0"),
        ("p/lancha x-line", "para lancha x-line"),
        ("S-TRONIC 3.3-3.8", "automatico 3.3-3.8"),
        ("USM-2450-74-ST", "usm-2450-74-st"),
        ("FULL - EXTRAFULL", "full extrafull"),
        ("DX/LX 4-5p", "dx_lx 4_5_puertas"),
        ("Adaptado a rural", "furgon_reformado"),
        ("ad. rural", "furgon_reformado"),
    ]

    df_ejemplos = pd.DataFrame(ejemplos, columns=['entrada', 'salida_esperada'])
    df_ejemplos['salida_obtenida'] = df_ejemplos['entrada'].apply(normalizar_texto)
    df_ejemplos['ok'] = df_ejemplos['salida_obtenida'] == df_ejemplos['salida_esperada']

    serie_real = pd.concat([
        df_bse['MODELO_ORIGINAL'].astype(str),
        df_autodata['MODELO_A_ORIGINAL'].astype(str)
    ], ignore_index=True).dropna().drop_duplicates()

    n = min(n_muestra, len(serie_real))
    df_revision = pd.DataFrame({'entrada_real': serie_real.sample(n=n, random_state=42)})
    df_revision['salida_normalizada'] = df_revision['entrada_real'].apply(normalizar_texto)

    return df_ejemplos, df_revision

print("Funciones de preprocesamiento definidas")
print(f"Alias de marcas configurados: {len(ALIAS_MARCAS)} reglas")

## 5️⃣ Preprocesamiento de datos

In [ ]:
print("Iniciando preprocesamiento de catálogos...")

# --- PROCESAR BSE ---
df_bse['marca_norm'] = df_bse['MARCA'].apply(normalizar_marca)
df_bse['modelo_norm'] = df_bse['MODELO_ORIGINAL'].apply(normalizar_texto)
df_bse[['anio_inicio', 'anio_fin']] = df_bse['RANGO_ANIOS'].apply(
    lambda x: pd.Series(extraer_anio_inicio_fin(x))
)
# Filtrar registros sin años procesables
df_bse = df_bse[df_bse['anio_inicio'].notna()].copy()
df_bse.reset_index(drop=True, inplace=True)  # IMPORTANTE: resetear índice para indexación consistente con embeddings

# --- PROCESAR AUTODATA ---
df_autodata['marca_norm'] = df_autodata['MARCA_A'].apply(normalizar_marca)
df_autodata['modelo_norm'] = df_autodata['MODELO_A_ORIGINAL'].apply(normalizar_texto)
# Año técnico para matching (derivado desde texto)
df_autodata['anio'] = pd.to_numeric(df_autodata['ANIO_A'], errors='coerce').fillna(9999).astype(int)

print("Preprocesamiento completado.")
print(f"BSE limpio: {len(df_bse):,} registros | Autodata: {len(df_autodata):,} registros.")

print("\nEjemplo de preprocesamiento:")
print("\nBSE:")
display(df_bse[['MARCA', 'marca_norm', 'MODELO_ORIGINAL', 'modelo_norm', 'RANGO_ANIOS', 'anio_inicio', 'anio_fin']].head(10))
print("\nAutodata:")
display(df_autodata[['MARCA_A', 'marca_norm', 'MODELO_A_ORIGINAL', 'modelo_norm', 'ANIO_A', 'anio']].head(10))

# Validación recomendada antes de ejecutar matching completo
df_validacion_ejemplos, df_revision_50 = validar_normalizacion_contextual(df_bse, df_autodata, n_muestra=50)
print("\nValidación de ejemplos controlados:")
display(df_validacion_ejemplos)
print("\nMuestra de 50 casos reales para revisión manual:")
display(df_revision_50)

## 6️⃣ Cargar Modelos y Construir Índices Optimizados

In [ ]:
# Detectar dispositivo para los modelos de IA (Transformers sí usarán GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando device para IA: {device}")

# ========== 1. MODELOS DE IA ==========
print("\n[1/4] Cargando modelos de embeddings y reranker...")
# BGE-M3 genera los vectores y Reranker decide el mejor candidato
model_bi = SentenceTransformer('BAAI/bge-m3', device=device)
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3', device=device)
print("Modelos cargados")

# ========== 2. ÍNDICE POR MARCA + AÑO (Pre-filtrado inteligente) ==========
print("\n[2/4] Construyendo índice de marca-año para filtrado rápido...")
# Diccionario: marca -> {año -> [índices]}
marca_anio_index = defaultdict(lambda: defaultdict(list))

for idx, row in df_bse.iterrows():
    marca = row['marca_norm']
    # Aseguramos que sean enteros
    try:
        anio_ini = int(row['anio_inicio'])
        anio_fin = int(row['anio_fin'])
        # Agregamos este registro a todos los años en su rango
        for year in range(anio_ini, anio_fin + 1):
            marca_anio_index[marca][year].append(idx)
    except (ValueError, TypeError):
        continue # Si hay datos sucios en años, los saltamos

print(f"Índice construido: {len(marca_anio_index)} marcas")

# ========== 3. EMBEDDINGS (Búsqueda vectorial) ==========
print("\n[3/4] Generando embeddings BGE-M3 para BSE...")
bse_textos = (df_bse['marca_norm'] + " " + df_bse['modelo_norm']).tolist()

# Generar embeddings (Esto usa GPU si está disponible, gracias a model_bi)
bse_embeddings = model_bi.encode(
    bse_textos,
    convert_to_tensor=False,  # numpy para dot-product directo
    batch_size=512,
    show_progress_bar=True,
    normalize_embeddings=True  # Normalizar para que Inner Product = Cosine Similarity
)

# Convertir a float32 para operaciones numéricas eficientes
bse_embeddings = bse_embeddings.astype('float32')

print(f"Embeddings generados: {bse_embeddings.shape[0]} vectores ({bse_embeddings.shape[1]}D)")

# ========== 4. ÍNDICES BM25 POR MARCA ==========
print("\n[4/4] Construyendo índices BM25 por marca...")
bm25_cache = {}
marca_to_indices = defaultdict(list)

# Agrupar índices por marca
for idx, row in df_bse.iterrows():
    marca_to_indices[row['marca_norm']].append(idx)

# Crear un BM25 por marca
for marca, indices in tqdm(marca_to_indices.items(), desc="BM25 por marca"):
    # Tokenizamos simplemente por espacios para BM25
    corpus_marca = [str(df_bse.loc[i, 'modelo_norm']).split() for i in indices]
    bm25_cache[marca] = {
        'bm25': BM25Okapi(corpus_marca),
        'indices': indices # Guardamos los índices reales de dataframe que corresponden al BM25 local
    }

print(f"{len(bm25_cache)} índices BM25 cacheados")
print("\n" + "="*70)
print("SISTEMA OPTIMIZADO LISTO")
print("="*70)

## 7️⃣ Función de Matching Optimizada

In [ ]:
def calcular_penalizacion_anio(anio_query, anio_ini_bse, anio_fin_bse):
    """
    Calcula penalización gradual basada en qué tan lejos está el año.
    Retorna multiplicador [0.7 a 1.0]
    """
    if anio_query == 9999:  # Año desconocido
        return 0.9  # Penalización leve por incertidumbre

    # Dentro del rango = sin penalización
    if anio_ini_bse <= anio_query <= anio_fin_bse:
        return 1.0

    # Calcular distancia mínima al rango
    if anio_query < anio_ini_bse:
        distancia = anio_ini_bse - anio_query
    else:
        distancia = anio_query - anio_fin_bse

    # Penalización: 1 año = 0.95, 2 años = 0.85, 3+ años = 0.7
    if distancia == 1:
        return 0.95
    elif distancia == 2:
        return 0.85
    else:
        return 0.7

def sigmoid_temperatura(x, temperatura=2.5):
    """Convierte logits a [0,1] con escala fija para comparabilidad entre registros."""
    return 1.0 / (1.0 + np.exp(-x / temperatura))

def encontrar_matches_optimizado(row_autodata, top_k=3, top_candidates=50):
    """
    Pipeline optimizado en 3 fases:
    1. Pre-filtrado inteligente (marca + año)
    2. Búsqueda híbrida (semántica directa + BM25 léxico)
    3. Reranking con cross-encoder
    """
    marca_auto = row_autodata['marca_norm']
    anio_auto = row_autodata['anio']
    modelo_auto = row_autodata['modelo_norm']
    query_text = f"{marca_auto} {modelo_auto}"

    # ========== FASE 1: PRE-FILTRADO INTELIGENTE ==========
    # 1.1 Resolver alias de marca (puede mapear a múltiples marcas BSE)
    marcas_bse = resolver_marcas_bse(marca_auto)

    # 1.1b Obtener candidatos por marca(s)
    indices_marca = []
    for m in marcas_bse:
        if m in marca_to_indices:
            indices_marca.extend(marca_to_indices[m])

    if not indices_marca:
        return []  # Ninguna marca resuelta existe en BSE

    # 1.2 Filtrar por año con ventana de ±2 años
    if anio_auto != 9999:
        indices_candidatos = set()
        for m in marcas_bse:
            if m in marca_anio_index:
                for offset in range(-2, 3):
                    year = anio_auto + offset
                    if year in marca_anio_index[m]:
                        indices_candidatos.update(marca_anio_index[m][year])

        indices_candidatos = list(indices_candidatos)

        # Si el filtro fue muy agresivo, usar todos de la marca
        if len(indices_candidatos) < 10:
            indices_candidatos = indices_marca
    else:
        indices_candidatos = indices_marca

    if not indices_candidatos:
        return []

    # ========== FASE 2: BÚSQUEDA HÍBRIDA ==========
    # 2.1 Búsqueda semántica directa sobre candidatos pre-filtrados
    query_embedding = model_bi.encode(
        [query_text],
        normalize_embeddings=True
    ).astype('float32')

    # Calcular similitud solo contra los candidatos pre-filtrados (evita truncamiento por k global)
    candidate_indices = np.array(indices_candidatos)
    candidate_embeddings = bse_embeddings[candidate_indices]
    similarities = np.dot(candidate_embeddings, query_embedding.T).flatten()

    # Tomar los mejores candidatos semánticos
    top_sem_count = min(top_candidates * 2, len(candidate_indices))
    top_sem_positions = np.argsort(similarities)[::-1][:top_sem_count]
    semantic_results = [(int(candidate_indices[pos]), float(similarities[pos])) for pos in top_sem_positions]

    if not semantic_results:
        return []

    # 2.2 Búsqueda léxica con BM25 (consulta en todas las marcas resueltas)
    bm25_dict = {}
    tokenized_query = modelo_auto.split()
    for m in marcas_bse:
        if m in bm25_cache:
            bm25_data = bm25_cache[m]
            bm25_model = bm25_data['bm25']
            bm25_indices_locales = bm25_data['indices']

            bm25_scores = bm25_model.get_scores(tokenized_query)

            # Normalizar BM25 dentro de cada marca independientemente
            if bm25_scores.max() > 0:
                bm25_scores = bm25_scores / bm25_scores.max()

            # Merge: conservar el mayor score por índice (para multi-marca)
            for idx, score in zip(bm25_indices_locales, bm25_scores):
                if idx not in bm25_dict or score > bm25_dict[idx]:
                    bm25_dict[idx] = score

    # 2.3 Combinar scores semántico + léxico + penalización de año
    combined_scores = []
    for idx, sem_score in semantic_results:
        lex_score = bm25_dict.get(idx, 0)

        # Penalización por diferencia de año
        anio_ini = df_bse.loc[idx, 'anio_inicio']
        anio_fin = df_bse.loc[idx, 'anio_fin']
        year_penalty = calcular_penalizacion_anio(anio_auto, anio_ini, anio_fin)

        # Score híbrido: (70% semántico + 30% léxico) × penalización_año (multiplicativa)
        hybrid_score = (0.70 * sem_score + 0.30 * lex_score) * year_penalty
        combined_scores.append((idx, hybrid_score))

    # Ordenar por score híbrido
    combined_scores.sort(key=lambda x: x[1], reverse=True)
    top_hybrid = combined_scores[:top_candidates]

    if not top_hybrid:
        return []

    # ========== FASE 3: RERANKING CON CROSS-ENCODER ==========
    indices_finales = [idx for idx, _ in top_hybrid]
    pairs = [
        [
            f"{query_text} {anio_auto if anio_auto != 9999 else ''}",
            f"{df_bse.loc[idx, 'marca_norm']} {df_bse.loc[idx, 'modelo_norm']} {int(df_bse.loc[idx, 'anio_inicio'])}-{int(df_bse.loc[idx, 'anio_fin'])}"
        ]
        for idx in indices_finales
    ]

    rerank_scores = reranker.predict(pairs)

    # SCORE LOCAL (ranking dentro del mismo registro)
    min_score = rerank_scores.min()
    max_score = rerank_scores.max()

    if max_score > min_score:
        normalized_scores = (rerank_scores - min_score) / (max_score - min_score)
    else:
        normalized_scores = np.ones_like(rerank_scores) * 0.5

    hybrid_scores_raw = np.array([score for _, score in top_hybrid], dtype=float)
    if hybrid_scores_raw.max() > 0:
        hybrid_scores_local = hybrid_scores_raw / hybrid_scores_raw.max()
    else:
        hybrid_scores_local = np.zeros_like(hybrid_scores_raw)

    final_scores = 0.8 * normalized_scores + 0.2 * hybrid_scores_local

    # SCORE_COMPARABLE (comparable entre registros)
    rerank_global = sigmoid_temperatura(rerank_scores, temperatura=2.5)
    hybrid_global = np.clip(hybrid_scores_raw, 0, 1)
    score_comparable = 0.85 * rerank_global + 0.15 * hybrid_global

    # Top K final
    top_k_indices = np.argsort(final_scores)[::-1][:top_k]

    return [{
        'ID_AUTODATA_BSE': df_bse.loc[indices_finales[i], 'ID_AUTODATA_BSE'],
        'score_comparable': round(float(score_comparable[i]), 4),
        'score': round(float(final_scores[i]), 4),
        'marca_bse': df_bse.loc[indices_finales[i], 'MARCA'],
        'modelo_bse': df_bse.loc[indices_finales[i], 'MODELO_ORIGINAL'],
        'rango_anios_bse': df_bse.loc[indices_finales[i], 'RANGO_ANIOS']
    } for i in top_k_indices]

print("Función de matching optimizada definida")

## 8️⃣ Ejecutar matching para todo el catálogo Autodata

In [ ]:
TOP_K = 3
resultados_finales = []

# Lookup O(1) para COMB y TIPO por ID_AUTODATA_BSE
bse_lookup = df_bse.drop_duplicates(subset='ID_AUTODATA_BSE').set_index('ID_AUTODATA_BSE')[['COMB', 'TIPO']].to_dict('index')

print(f"Iniciando matching de {len(df_autodata):,} registros...")
print("Estimado: ~120-150 minutos en T4 GPU\n")

# Procesamiento con barra de progreso
for idx, row in tqdm(df_autodata.iterrows(), total=len(df_autodata), desc="Matching"):
    matches = encontrar_matches_optimizado(row, top_k=TOP_K)

    # Estructura base homogénea para todas las filas
    res = {
        'ID_AUTADATA': row['ID_AUTADATA'],
        'MARCA_A': row['MARCA_A'],
        'MODELO_A_ORIGINAL': row['MODELO_A_ORIGINAL'],
        'ANIO_A': row['ANIO_A'],

        'SCORE_COMPARABLE': 0.0,
        'SCORE_1': 0.0,
        'SCORE_2': 0.0,
        'SCORE_3': 0.0,

        'ID_AUTODATA_BSE_1': 'NO_MATCH',
        'ID_AUTODATA_BSE_2': '',
        'ID_AUTODATA_BSE_3': '',

        'MARCA_BSE_1': '',
        'MARCA_BSE_2': '',
        'MARCA_BSE_3': '',

        'MODELO_BSE_1': '',
        'MODELO_BSE_2': '',
        'MODELO_BSE_3': '',

        'RANGO_ANIOS_BSE_1': '',
        'RANGO_ANIOS_BSE_2': '',
        'RANGO_ANIOS_BSE_3': '',

        'COMB_BSE': '',
        'TIPO_BSE': ''
    }

    if matches:
        res['SCORE_COMPARABLE'] = matches[0].get('score_comparable', 0.0)

        # Llenar con los Top K matches encontrados
        for i, m in enumerate(matches[:TOP_K]):
            suffix = f"_{i+1}"
            res[f'SCORE{suffix}'] = m['score']
            res[f'ID_AUTODATA_BSE{suffix}'] = m['ID_AUTODATA_BSE']
            res[f'MARCA_BSE{suffix}'] = m['marca_bse']
            res[f'MODELO_BSE{suffix}'] = m['modelo_bse']
            res[f'RANGO_ANIOS_BSE{suffix}'] = m['rango_anios_bse']

        # Campos de trazabilidad solo para el primer match
        first_match_bse_id = matches[0]['ID_AUTODATA_BSE']

        # Verificar si el ID es válido (string no vacío) y buscar en lookup O(1)
        if first_match_bse_id and not pd.isna(first_match_bse_id) and str(first_match_bse_id).strip() != '':
            bse_id_str = str(first_match_bse_id)
            if bse_id_str in bse_lookup:
                res['COMB_BSE'] = bse_lookup[bse_id_str].get('COMB', '')
                res['TIPO_BSE'] = bse_lookup[bse_id_str].get('TIPO', '')

    resultados_finales.append(res)

# Crear DataFrame
df_resultados = pd.DataFrame(resultados_finales)

print("\n" + "="*70)
print("MATCHING COMPLETADO")
print("="*70)
print(f"Total procesado: {len(df_resultados):,} registros")
print(f"Sin matches: {(df_resultados['SCORE_1'] == 0).sum():,}")
print(f"Con matches: {(df_resultados['SCORE_1'] > 0).sum():,}")
print(f"\nScore promedio: {df_resultados[df_resultados['SCORE_1'] > 0]['SCORE_1'].mean():.3f}")

display(df_resultados.head(10))

## 9️⃣ Análisis de Calidad y Exportación

In [ ]:
print("\nDISTRIBUCIÓN DE SCORE_COMPARABLE (GLOBAL):\n")
df_con_match_global = df_resultados[df_resultados['SCORE_COMPARABLE'] > 0]

print(f"Score >= 0.9 (Alta confianza):  {(df_con_match_global['SCORE_COMPARABLE'] >= 0.9).sum():,}")
print(f"Score 0.7-0.9 (Media confianza): {((df_con_match_global['SCORE_COMPARABLE'] >= 0.7) & (df_con_match_global['SCORE_COMPARABLE'] < 0.9)).sum():,}")
print(f"Score 0.5-0.7 (Revision humana):         {((df_con_match_global['SCORE_COMPARABLE'] >= 0.5) & (df_con_match_global['SCORE_COMPARABLE'] < 0.7)).sum():,}")
print(f"Score < 0.5 (Baja confianza):    {(df_con_match_global['SCORE_COMPARABLE'] < 0.5).sum():,}")

# Exportar resultados
archivo_salida = 'resultados_matching_completo_v3.csv'
df_resultados.to_csv(archivo_salida, index=False, encoding='utf-8-sig')

print(f"\nResultados exportados a: {archivo_salida}")

print("\n" + "="*70)
print("========= PROCESO COMPLETADO EXITOSAMENTE =========")
print("="*70)

In [ ]:
columnas_origen = [
    'ID_AUTADATA',
    'ID_AUTODATA_BSE_1',
    'SCORE_COMPARABLE',
    'SCORE_1',
    'MARCA_A',
    'MARCA_BSE_1',
    'ANIO_A',
    'RANGO_ANIOS_BSE_1',
    'MODELO_A_ORIGINAL',
    'MODELO_BSE_1',
    'COMB_BSE',
    'TIPO_BSE'
]

renombrar_columnas = {
    'ID_AUTADATA': 'ID_AUTODATA',
    'ID_AUTODATA_BSE_1': 'ID_AUTODATA_BSE',
    'SCORE_1': 'SCORE_LOCAL',
    'MARCA_BSE_1': 'MARCA_BSE',
    'RANGO_ANIOS_BSE_1': 'RANGO_ANIOS_BSE',
    'MODELO_A_ORIGINAL': 'MODELO_A',
    'MODELO_BSE_1': 'MODELO_BSE'
}

df_filtrado = df_resultados[columnas_origen].rename(columns=renombrar_columnas).copy()
print("DataFrame 'df_filtrado' creado con columnas finales estandarizadas.")
display(df_filtrado.head(10))

archivo_filtrado_salida = "resultados_matching_resumido_v3.csv"
df_filtrado.to_csv(archivo_filtrado_salida, index=False, encoding="utf-8-sig")

print(f"DataFrame filtrado exportado a: {archivo_filtrado_salida}")
